In [3]:
%%capture
%pip install sentence_transformers

In [19]:
import numpy as np
import pandas as pd

In [5]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
model_name = 'multi-qa-mpnet-base-dot-v1'

In [6]:
url = f'{github_url}?raw=1'
df = pd.read_csv(url)
df = df.iloc[:300]

In [7]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
answer_llm = df.iloc[0].answer_llm
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [12]:
embedding_model.encode(answer_llm)[0]

-0.42244658

In [14]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [13]:
df_dict = df.to_dict(orient="records")

In [15]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']

    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)

    return v_llm.dot(v_orig)

In [16]:
from tqdm import tqdm

In [17]:
similarity = []

for record in tqdm(df_dict):
    sim = compute_similarity(record)
    similarity.append(sim)

100%|██████████| 300/300 [03:44<00:00,  1.34it/s]


In [18]:
df['cosine'] = similarity
df['cosine'].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547925
25%       24.307846
50%       28.336873
75%       31.674308
max       39.476013
Name: cosine, dtype: float64

In [20]:
def normalize_vector(v):
  norm = np.sqrt((v * v).sum())
  v_norm = v / norm
  return v_norm

def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']

    v_llm = normalize_vector(embedding_model.encode(answer_llm))
    v_orig = normalize_vector(embedding_model.encode(answer_orig))

    return v_llm.dot(v_orig)

In [21]:
similarity = []

for record in tqdm(df_dict):
    sim = compute_similarity(record)
    similarity.append(sim)

100%|██████████| 300/300 [03:42<00:00,  1.35it/s]


In [22]:
df['cosine'] = similarity
df['cosine'].describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine, dtype: float64

In [23]:
%%capture
%pip install rouge

In [26]:
df.iloc[10]

answer_llm     Yes, all sessions are recorded, so if you miss...
answer_orig    Everything is recorded, so you won’t miss anyt...
document                                                5170565b
question                    Are sessions recorded if I miss one?
course                                 machine-learning-zoomcamp
cosine                                                  0.777956
Name: 10, dtype: object

In [29]:
from rouge import Rouge
rouge_scorer = Rouge()

score = rouge_scorer.get_scores(df.iloc[10]['answer_llm'], df.iloc[10]['answer_orig'])[0]

In [30]:
score

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [33]:
rouge_1_f = score['rouge-1']['f']
rouge_2_f = score['rouge-2']['f']
rouge_l_f = score['rouge-l']['f']

avg_f = (rouge_1_f + rouge_2_f + rouge_l_f) / 3
print(avg_f)

0.35490034990035496


In [34]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df['answer_llm'], df['answer_orig'])

In [43]:
rouge_2_f_scores = [entry['rouge-2']['f'] for entry in scores]

In [44]:
np.mean(rouge_2_f_scores)

0.20696501983423318